# Implementing distance constraint in ER
* We will not compare sequences i and j if |i-j| < 5

In [2]:
import ecc_tools as tools
import numpy as np
import pandas as pd
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
#import emachine as EM
from direct_info import direct_info

import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

from scipy.sparse import csr_matrix
from joblib import Parallel, delayed
import timeit

import matplotlib.pyplot as plt
%matplotlib inline

import sys
import numpy as np
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
import expectation_reflection as ER
from direct_info import direct_info
from joblib import Parallel, delayed

tools.hide_toggle()

### Read in Data

In [4]:
# Read in Protein structure
data_path = '../Pfam-A.full'
pfam_id = 'PF04542'
pfam_id = 'PF00186'
pdb = np.load('%s/%s/pdb_refs.npy'%(data_path,pfam_id))

# Pre-Process Structure Data
# delete 'b' in front of letters (python 2 --> python 3)
pdb = np.array([pdb[t,i].decode('UTF-8') for t in range(pdb.shape[0]) \
         for i in range(pdb.shape[1])]).reshape(pdb.shape[0],pdb.shape[1])

# Print number of pdb structures in Protein ID folder
npdb = pdb.shape[0]
print('number of pdb structures:',npdb)

# Print PDB array 
print(pdb)
print(pdb[0])

# Create pandas dataframe for protein structure
df = pd.DataFrame(pdb,columns = ['PF','seq','id','uniprot_start','uniprot_start',\
                                 'pdb_id','chain','pdb_start','pdb_end'])
df.head()

ipdb = 0
print('seq:',int(pdb[ipdb,1]))

from data_processing import data_processing
# data processing
s0,cols_removed, s_index = data_processing(data_path,pfam_id,ipdb,\
                gap_seqs=0.2,gap_cols=0.2,prob_low=0.004,conserved_cols=0.9)

print("Removed columns: ",cols_removed)

tools.hide_toggle()

number of pdb structures: 372
[['PF00186' '69' 'Q5KZ26_GEOKA' ... 'B' '1' '160']
 ['PF00186' '69' 'Q5KZ26_GEOKA' ... 'A' '1' '160']
 ['PF00186' '83' 'Q81R22_BACAN' ... 'B' '2' '160']
 ...
 ['PF00186' '6952' 'DYR_MYCTU' ... 'A' '1' '158']
 ['PF00186' '7457' 'Q834R2_ENTFA' ... 'A' '1' '161']
 ['PF00186' '7457' 'Q834R2_ENTFA' ... 'A' '1' '161']]
['PF00186' '69' 'Q5KZ26_GEOKA' '1' '160' '1ZDR' 'B' '1' '160']
seq: 69
shape of s (import from msa.npy):
 (7750, 918)
shape of s (after UTF-8 decode):
 (7750, 918)
pdb:
 [[b'PF00186' b'69' b'Q5KZ26_GEOKA' ... b'B' b'1' b'160']
 [b'PF00186' b'69' b'Q5KZ26_GEOKA' ... b'A' b'1' b'160']
 [b'PF00186' b'83' b'Q81R22_BACAN' ... b'B' b'2' b'160']
 ...
 [b'PF00186' b'6952' b'DYR_MYCTU' ... b'A' b'1' b'158']
 [b'PF00186' b'7457' b'Q834R2_ENTFA' ... b'A' b'1' b'161']
 [b'PF00186' b'7457' b'Q834R2_ENTFA' ... b'A' b'1' b'161']]
pdb (after UTF-8 decode, removing 'b'):
 [['PF00186' '69' 'Q5KZ26_GEOKA' ... 'B' '1' '160']
 ['PF00186' '69' 'Q5KZ26_GEOKA' ... 'A' '1

In [5]:
# number of positions
n_var = s0.shape[1]
print("Number of residue positions:",n_var)

# number of aminoacids at each position
mx = np.array([len(np.unique(s0[:,i])) for i in range(n_var)])
#mx = np.array([m for i in range(n_var)])
print("Number of different amino acids at each position",mx)

mx_cumsum = np.insert(mx.cumsum(),0,0)
i1i2 = np.stack([mx_cumsum[:-1],mx_cumsum[1:]]).T 
print("(Sanity Check) Column indices of first and (",i1i2[0],") and last (",i1i2[-1],") positions")
print("(Sanity Check) Column indices of second and (",i1i2[1],") and second to last (",i1i2[-2],") positions")


# number of variables
mx_sum = mx.sum()
print("Total number of variables",mx_sum)

# number of bias term
n_linear = mx_sum - n_var

tools.hide_toggle()

Number of residue positions: 137
Number of different amino acids at each position [ 6 16 12  7  8 16  8 17 12 14  9 19 11 10 13  5  5 14  9  6 11  2  9 12
 11  9 10 16 17 11 12 13  5  4  9  7 11  6  5 10 12 10  9  9 17  8  8  2
  5  2 11 11 18 18 20 20 20 14 13 15 16 15 14 14  7 12 15 12  8 15 15 21
 19 16 16 20 15 17 13 11  8 10 12 13  4  3 13 15 12 11 13 19  9 11 17  7
 14  8 16  3 16 19 15 14 15  7  7 15  7 17 10 16 20 17 18  5 17 16 17 20
 19 18 21 21 19 15 17 13 16 13 14 10 14  9 18 14  5]
(Sanity Check) Column indices of first and ( [0 6] ) and last ( [1697 1702] ) positions
(Sanity Check) Column indices of second and ( [ 6 22] ) and second to last ( [1683 1697] ) positions
Total number of variables 1702


In [6]:
onehot_encoder = OneHotEncoder(sparse=False,categories='auto')
# s is OneHot encoder format, s0 is original sequnce matrix
s = onehot_encoder.fit_transform(s0)
#print("Amino Acid sequence Matrix\n",s0)
#print("OneHot sequence Matrix\n",s)
#print("An individual element of the OneHot sequence Matrix (size:",
#      s.shape,") --> ",s[0], " has length ",s[0].shape)

tools.hide_toggle()

### First lets consider no distance constraint.
* Note: We have added a new output to data_processing: s_index 
    * Original index of processed referenece sequence data

In [23]:
# Define wight matrix with variable for each possible amino acid at each sequence position
w = np.zeros((mx.sum(),mx.sum())) 
h0 = np.zeros(mx.sum())

# Expectation Reflection
# Expectation Reflection
#=========================================================================================
def predict_w(s,i0,i1i2,niter_max,l2):
    #print('i0:',i0)
    i1,i2 = i1i2[i0,0],i1i2[i0,1]

    x = np.hstack([s[:,:i1],s[:,i2:]])
    y = s[:,i1:i2]

    h01,w1 = ER.fit(x,y,niter_max,l2)

    return h01,w1

#-------------------------------
# parallel
start_time = timeit.default_timer()
#res = Parallel(n_jobs = 4)(delayed(predict_w)\
#res = Parallel(n_jobs = 8)(delayed(predict_w)\
res = Parallel(n_jobs = 32)(delayed(predict_w)\
        (s,i0,i1i2,niter_max=10,l2=100.0)\
        for i0 in range(n_var))

run_time = timeit.default_timer() - start_time
print('run time:',run_time)
#----------------niter_max,l2)
for i0 in range(n_var):
    i1,i2 = i1i2[i0,0],i1i2[i0,1]

    h01 = res[i0][0]
    w1 = res[i0][1]

    h0[i1:i2] = h01
    w[:i1,i1:i2] = w1[:i1,:]
    w[i2:,i1:i2] = w1[i1:,:]

# make w symmetric
w = (w + w.T)/2.

# Calculate Direct Information from infered weights and Original Sequence Data
di = direct_info(s0,w)

# Save w and di for future use
np.save('ER_w.npy',w)
np.save('ER_DI.npy',di)

tools.hide_toggle()

/Users/cresswellclayec/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


run time: 320.95470380200004


In [24]:
# Calculate the direct info sans distance restraint
di_regular = direct_info(s0,w)

### Now constrain |i-j| > 5.

In [42]:
# Define wight matrix with variable for each possible amino acid at each sequence position
w = np.zeros((mx.sum(),mx.sum())) 
h0 = np.zeros(mx.sum())

# Expectation Reflection
# Expectation Reflection
#=========================================================================================

#------------- Define Distance Bounds --------------#
iL = np.zeros(s_index.shape,dtype=int)
iR = np.zeros(s_index.shape,dtype=int)
if s0.shape[1] != s_index.shape[0]:
    print("Print original index array size does not match sequence array size")
    exit(0);

print("s_index (length=%d):\n"%s_index.shape[0],s_index)
for i00, r_index in enumerate(s_index):
    #print("deining bounds for i0 = %d"%i0 0)
    # find left bound
    for ii in range(i00):
        #print("for %d, |%d - %d| = %d"%(ii,r_index,s_index[ii],abs(r_index-s_index[ii])))
        if abs(r_index - s_index[ii]) > 5:
            iL[i00] = ii
    # find right bound
    for ii in range(s_index.shape[0]-1,i00,-1):
        #print("for %d, |%d - %d| = %d"%(ii,r_index,s_index[ii],abs(r_index-s_index[ii])))
        if abs(r_index - s_index[ii]) > 5:
            iR[i00] = ii
print("Left and Right bounds: ",iL,iR)
#---------------------------------------------------#

def predict_w(s,i0,iL,iR,i1i2,niter_max,l2):

    iL,iR = i1i2[iL,0],i1i2[iR,1]
    i1,i2 = i1i2[i0,0],i1i2[i0,1]
    
    x = np.hstack([s[:,:iL],s[:,iR:]])
    y = s[:,i1:i2]
    h01,w1 = ER.fit(x,y,niter_max,l2)

    return h01,w1


#-------------------------------
# parallel
start_time = timeit.default_timer()
#res = Parallel(n_jobs = 4)(delayed(predict_w)\
#res = Parallel(n_jobs = 8)(delayed(predict_w)\
res = Parallel(n_jobs = 32)(delayed(predict_w)\
        (s,i0_restr,iL,iR,i1i2,niter_max=10,l2=100.0)\
        for i0_restr in range(n_var))

run_time = timeit.default_timer() - start_time
print('run time:',run_time)
#----------------niter_max,l2)
for i0 in range(n_var):
    i1,i2 = i1i2[i0,0],i1i2[i0,1]

    h01 = res[i0][0]
    w1 = res[i0][1]

    h0[i1:i2] = h01
    w[:i1,i1:i2] = w1[:i1,:]
    w[i2:,i1:i2] = w1[i1:,:]

# make w symmetric
w = (w + w.T)/2.

# Calculate Direct Information from infered weights and Original Sequence Data
di = direct_info(s0,w)

# Save w and di for future use
np.save('ER_w.npy',w)
np.save('ER_DI.npy',di)

tools.hide_toggle()

s_index (length=137):
 [  1   2   3   4   5   7   8   9  10  11  12  15  16  17  18  19  20  22
  23  24  25  26  27  28  29  31  32  33  35  36  37  38  39  40  44  46
  47  48  49  50  51  52  54  55  57  58  59  60  61  62  63  64  65  66
  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
  85  86  87  88  90  91  92  93  94  95  98  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 115 116 117 118 119 120 121 122 124 125
 126 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 146
 147 148 149 150 151 152 153 154 155 156 157]
Left and Right bounds:  [  0   0   0   0   0   0   1   2   3   4   4   7   8   9  10  10  10  12
  13  14  15  16  16  17  18  20  21  22  24  24  25  26  27  27  31  33
  33  33  33  34  34  35  37  38  40  41  41  42  43  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  71  72  73  74  75  75  78  79  80  81  81  81  82  83
  84  85  86  87  88  89  90  9

TypeError: only integer scalar arrays can be converted to a scalar index

## Compare Optimal DCA and ER (ct_thresh = 1.5 and 2. resp.)
* Taking the optimal threshold for each gives comparable accuracy
* ER Has higher accuracy for lower threshold

In [ ]:
# find optimal threshold of distance for both DCA and ER
ct_thres = np.linspace(1.5,10.,18,endpoint=True)
n = ct_thres.shape[0]

auc_ER_reg = np.zeros(n)
auc_ER_restr = np.zeros(n)
for i in range(n):
    p,tp,fp = tools.roc_curve(ct,di_regular,ct_thres[i])
    auc_ER_reg[i] = tp.sum()/tp.shape[0]
    
    p,tp,fp = tools.roc_curve(ct,di_restr,ct_thres[i])
    auc_ER_restr[i] = tp.sum()/tp.shape[0]
    
i0_reg = np.argmax(auc_ER_reg)
i0_restr = np.argmax(auc_ER_restr)

print('regular ER auc max:',ct_thres[i0_reg],auc[i0_reg])
p0_reg,tp0_reg,fp0_reg = tools.roc_curve(ct,di_regular,ct_thres[i0_reg])
print('restrained ER auc max:',ct_thres[i0_restr],auc[i0restr])
p0_restr,tp0_restr,fp0_restr = tools.roc_curve(ct,di_restr,ct_thres[i0_restr])

tools.hide_toggle()

In [ ]:
# Plot ROC for optimal distally constrained vs versus un-constrained ER
plt.subplot2grid((1,3),(0,0))
plt.title('ROC at thres (regular ER, restrained ER)\n = (%3.2f, %3.2f)'%(ct_thres[i0_reg],ct_thres[i0_restr]))
plt.plot(fp0_restr,tp0_restr,'b-',label="Restrained ER")
plt.plot(fp0_reg,tp0_reg,'r-',label="Regular ER")
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

# Plot AUC for DCA and ER
plt.subplot2grid((1,3),(0,1))
plt.title('AUC (regular ER, restrained ER) max\n = (%3.2f, %3.2f)' %(auc_DCA[i0_reg], auc_ER[i0_restr]))
plt.plot([ct_thres.min(),ct_thres.max()],[0.5,0.5],'k--')
plt.plot(ct_thres,auc_reg,'b-',label="Restrained ER")
plt.plot(ct_thres,auc_restr,'r-',label="Regular ER")
plt.xlim([ct_thres.min(),ct_thres.max()])
plt.ylim([0,auc.max()+0.1])
plt.xlabel('distance threshold')
plt.ylabel('AUC')
plt.legend()

# Plot Precision of optimal DCA and ER
plt.subplot2grid((1,3),(0,2))
plt.title('Precision at thres \n(regular ER, restrained ER) = (%3.2f, %3.2f)'%(ct_thres[i0_reg],ct_thres[i0_restr]))
plt.plot( p0_reg,tp0_reg / (tp0_reg + fp0_reg),'r-',label='Regular thres = %s'%ct_thres[i0_reg])
plt.plot( p0_restr,tp0_restr / (tp0_restr + fp0_restr),'b-',label='Restrained thres = %s'%ct_thres[i0_restr])
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

plt.tight_layout(h_pad=1, w_pad=1.5)


tools.hide_toggle()